# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [44]:
import string
import math
from enum import Enum

## Setting Token Enum for testing purpose

In [45]:
class TokenTypes(Enum):
    # Data
    INTEGER = int

    # Aritmetics
    PLUS = "+"
    MINUS = "-"
    MULTIPLY = "*"
    DIVIDE = "/"

    # Mehtods
    FOR = "for"
    LBRACKET = "("
    RBRACKET = ")"

    # Else
    EOF = None

In [43]:
list(TokenTypes)

[<TokenTypes.INTEGER: <class 'int'>>,
 <TokenTypes.PLUS: '+'>,
 <TokenTypes.MINUS: '-'>,
 <TokenTypes.MULTIPLY: '*'>,
 <TokenTypes.DIVIDE: '/'>,
 <TokenTypes.FOR: 'for'>,
 <TokenTypes.LBRACKET: '('>,
 <TokenTypes.RBRACKET: ')'>,
 <TokenTypes.EOF: None>]

In [46]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value

## Lexer

In [49]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]
    
    # moves the pointer a character forward
    def forward(self, index) -> tuple[str,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None, index
        
        return self.input[index],index
    
    def future_token(self, index) -> tuple[Token,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None,index
        
        return self.get_token(self.input[index], index)
    
    def get_int(self, index: int) -> tuple[int,int]:
        result = ''

        # checks if there are multiple digits
        while True:
            if index < len(self.input) and self.input[index] != None and self.input[index].isnumeric():
                result += self.input[index]

                if index + 1 < len(self.input) and self.input[index + 1].isnumeric():
                    _,index = self.forward(index)
                else:
                    break
            else:
                break

        return int(result),index
    
    def get_token(self, char: string, index: int) -> tuple[Token,int]:
        # checks if there is no more character.
        if char is None:
            return Token(TokenTypes.EOF, None),index
        
        # skip spaces.
        if char == ' ':
            char,index = self.forward(index)
            return self.get_token(char, index)

        # checks if the current character is a number.
        if char.isnumeric():
            result,index = self.get_int(index)
            return Token(TokenTypes.INTEGER, result),index

        # checks if the current character is a supported token type.
        match char:
            case TokenTypes.DIVIDE.value:
                return Token(TokenTypes.DIVIDE, TokenTypes.DIVIDE.value),index
            case TokenTypes.MULTIPLY.value:
                return Token(TokenTypes.MULTIPLY, TokenTypes.MULTIPLY.value),index
            case TokenTypes.PLUS.value:
                return Token(TokenTypes.PLUS, TokenTypes.PLUS.value),index
            case TokenTypes.MINUS.value:
                return Token(TokenTypes.MINUS, TokenTypes.MINUS.value),index
            case TokenTypes.LBRACKET.value:
                return Token(TokenTypes.LBRACKET, TokenTypes.LBRACKET.value),index
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value),index  
                  
        return Token(TokenTypes.EOF, None),index

## Testing Lexer Types

In [50]:
lexer = lexicon("723 + 2 - 5 * 1235 / 100")

while lexer.current_char is not None:
    token,lexer.index = lexer.get_token(lexer.current_char, lexer.index)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    future_token,index = lexer.future_token(lexer.index)
    if future_token is not None:
        print("Future:" + str(future_token.type) + "\n")
    lexer.current_char, lexer.index = lexer.forward(lexer.index)

723 is of the tokentype: TokenTypes.INTEGER
Future:TokenTypes.PLUS

+ is of the tokentype: TokenTypes.PLUS
Future:TokenTypes.INTEGER

2 is of the tokentype: TokenTypes.INTEGER
Future:TokenTypes.MINUS

- is of the tokentype: TokenTypes.MINUS
Future:TokenTypes.INTEGER

5 is of the tokentype: TokenTypes.INTEGER
Future:TokenTypes.MULTIPLY

* is of the tokentype: TokenTypes.MULTIPLY
Future:TokenTypes.INTEGER

1235 is of the tokentype: TokenTypes.INTEGER
Future:TokenTypes.DIVIDE

/ is of the tokentype: TokenTypes.DIVIDE
Future:TokenTypes.INTEGER

100 is of the tokentype: TokenTypes.INTEGER


## Syntax Tree Nodes

In [51]:
class BaseNode:
    def __init__(self, token) -> None:
        self.token = token
    
    def visit(self, node):
        if isinstance(node, OperatorNode):
            return self.visit_operatorNode(node)
        
        return self.visit_numberNode(node)
        
    def visit_operatorNode(self, node):
        match node.token.type:
            case TokenTypes.DIVIDE:
                return self.visit(node.leftNode) // self.visit(node.rightNode)
            case TokenTypes.MULTIPLY:
                return self.visit(node.leftNode) * self.visit(node.rightNode)
            case TokenTypes.PLUS:
                return self.visit(node.leftNode) + self.visit(node.rightNode)
            case TokenTypes.MINUS:
                return self.visit(node.leftNode) - self.visit(node.rightNode)   

    def visit_numberNode(self, node):
        return node.value

class NumberNode(BaseNode):
    def __init__(self, token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

## Parser

In [52]:
class Parser:
    def __init__(self, lexer: lexicon) -> None:
        self.lexer = lexer
        self.current_token, self.index = lexer.get_token(self.lexer.current_char, self.lexer.index)
    
    def forward(self) -> None:
        self.lexer.current_char, self.lexer.index = self.lexer.forward(self.index)
        self.current_token, self.index = lexer.get_token(self.lexer.current_char, self.lexer.index)
    
    def klapu_STRI(self) -> BaseNode:
        leftNode = self.kla_PU_stri()
        token = self.current_token

        if token.type == TokenTypes.PLUS or token.type == TokenTypes.MINUS:
            self.forward()
        
            rightNode = self.kla_PU_stri()
            return OperatorNode(token, leftNode, rightNode)
        
        return leftNode

    def kla_PU_stri(self) -> BaseNode:
        leftNode = self.KLA_pustri()
        token = self.current_token

        if token.type == TokenTypes.MULTIPLY or token.type == TokenTypes.DIVIDE:
            self.forward()
        
            rightNode = self.KLA_pustri()
            return OperatorNode(token, leftNode,rightNode)
        
        return leftNode
    

    def KLA_pustri(self) -> BaseNode:
        token = self.current_token

        if token.type == TokenTypes.INTEGER:
            self.forward()
            return NumberNode(token)
        
        if token.type == TokenTypes.LBRACKET:
            self.forward()
            node = self.expr()
            self.forward()
            return node
        
        return token
    
    def expr(self) -> BaseNode:
        return self.klapu_STRI()
    
    def parse(self) -> BaseNode:
        return self.expr()

In [53]:
class Interpreter:
    def __init__(self, parser: Parser):
        self.parser = parser

    def interpret(self):
        tree = self.parser.parse()
        return tree.visit(tree)

In [54]:
text = "(2 * 3 + 2) * (2 + 4)"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

48